In [6]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import batman
from scipy.optimize import least_squares
import emcee

Given some time-series data, which are the results of photometry (in a previous step), I want to:
 - use scipy.optimize to create a fast fit which will give me errors
 - use these error estimates to perform an MCMC fit

We want to use the weighted linear least-squares fit method instead of some scipy tool:
https://en.wikipedia.org/wiki/Linear_least_squares_(mathematics)

*Issues:*
 - What is the linear light curve equation from Saul Rappaport?
 - How tf do classes work?
 - How many free variables are we assuming? Just Rp and Baseline?

In [4]:
def DataPrep(dir_inp,planet):
    
    "First, read in the data from an AIJ output table"
    data = pandas.read_csv(dir_inp)
    flux = data['rel_flux_T1']
    JD = data['JD-OBS']
    BJD = data['BJD-OBS']
    error = data['rel_flux_err_T1']
    
    "Plot the lightcurve to choose a masked region"
    plt.figure()
    plt.scatter(JD,flux)
    plt.errorbar(JD,flux,yerr=error,fmt='o')
    plt.title(planet)
    plt.xlabel('JD')
    plt.ylabel('Relative Flux (unitless)')
    plt.show()
    
    low = raw_input("When is the start of transit? ")
    high = raw_input("When is the end of transit? ")
    mask = np.where(np.logical_not((date >= low)& (date <= high)))[0]
    
    def f_x(x,a,b):
        return a + b*x

    popt, pcov = curve_fit(f_x,date[mask],flux[mask])

    print("Polynomial parameters:",popt)

    poly = f_x(date[mask], *popt)

    baseline_fit = f_x(date, *popt)
    
    normed_flux = flux/baseline_fit
    normed_err = error/baseline_fit

    plt.scatter(JD,normed_flux)
    plt.errorbar(JD,normed_flux,yerr=normed_err,fmt='o')
    plt.xlabel('Hours')
    plt.ylabel('Normalized Flux')
    plt.title(planet)
    plt.xlabel('JD')
    plt.ylabel('Relative Flux (unitless)')
    plt.show()
    
    return JD
    return normed_flux

In [3]:
def Fast_Fit(date,flux):
    
    
    def BATMAN(Rp,Baseline,period,a,t=date):
        params = batman.TransitParams()
        params.t0 = 0.                       #time of inferior conjunction
        params.per = period                  #period in hours
        params.rp = Rp                       #planet radius (in units of stellar radii)
        params.a = a                         #semi-major axis (in units of stellar radii)
        params.inc = 90.                     #orbital inclination (in degrees)
        params.ecc = 0.                      #eccentricity
        params.w = 90.                       #longitude of periastron (in degrees)
        params.u = [0.1956, 0.3700]          #limb darkening coefficients [u1, u2]
        params.limb_dark = "quadratic"       #limb darkening model
        
        m = batman.TransitModel(params, t)    #initializes model
    
        flux = m.light_curve(params)*Baseline        #calculates light curve
        return flux
    
    results = least_squares(fun=BATMAN,x0=[0.01,1,30,16])
    model_params = results[0]
    residuals = results[2]
    RMS = np.sqrt(np.mean(residuals**2.0))
    
    return RMS

In [ ]:
def run_emcee(planet,Model=True,Plot=True):
    
    def BATMAN_MODEL(Baseline, Rp, t = None):
        params = batman.TransitParams()
        params.t0 = 0.                       #time of inferior conjunction
        params.per = 39.09432                #period in hours
        params.rp = Rp                       #planet radius (in units of stellar radii)
        params.a = 16.                       #semi-major axis (in units of stellar radii)
        params.inc = 90.                     #orbital inclination (in degrees)
        params.ecc = 0.                      #eccentricity
        params.w = 90.                       #longitude of periastron (in degrees)
        params.u = [0.1956, 0.3700]          #limb darkening coefficients [u1, u2]
        params.limb_dark = "quadratic"       #limb darkening model
        
        m = batman.TransitModel(params, t)    #initializes model
    
        flux = m.light_curve(params)*Baseline        #calculates light curve
        return flux 
    
    if Model:
        # intialize some walkers
        ndim, nwalkers, nsteps, burnin = 2, 100, 10000, 1000

        # these are initial parameters
        Base_initial = np.random.uniform(0, 1, nwalkers)
        Rp_initial = np.random.uniform(0.1, 0.5, nwalkers)

        p0 = np.transpose([Base_initial, Rp_initial])
    
        # create a sampler and run it
        print("Now running emcee")
        sampler_gauss = emcee.EnsembleSampler(nwalkers, ndim, lnprob_gauss)
        result_gauss = sampler_gauss.run_mcmc(p0, nsteps)
    
        Base_gauss, Rp_gauss = sampler_gauss.chain.T[:, burnin:nsteps, :]

        sig1_Rp_gauss = np.percentile(Rp_gauss, [16., 50., 84.])
        print(sig1_Rp_gauss)
        sig1_Base_gauss = np.percentile(Base_gauss, [16., 50., 84.])
        print(sig1_Base_gauss)

        lnprob_gauss(parameters=[sig1_Base_gauss[1],sig1_Rp_gauss[1]],plot=True)
        gauss_rp_hist,gauss_bins = np.histogram(Rp_gauss,bins=60,normed=True)
    
    if Plot:
        "Plot the distributions"
        plt.figure()
        plt.plot(gauss_bins[:-1],gauss_rp_hist,label='Gaussian PDF')
        plt.title('Rp/R* Distributions')
        plt.legend(frameon=False)
        plt.xlabel('Rp/R*')
        plt.ylabel('Posterior Distrubtions')
        plt.savefig('{}_RP_Posteriors.pdf'.format(planet))
        plt.show()
    
    Posterior_bins['{} gauss'.format(planet)] = gauss_bins
    Posteriors['{} gauss'.format(planet)] = gauss_rp_hist

In [ ]:
def lnprob_gauss(parameters, plot=False, planet):
        
    times = highres_times
        
    # pull out some model parameters
    Baseline, Rp = parameters
    
    model = BATMAN_MODEL(Baseline, Rp, t = data_times)
    #This assumes the batman model is in photons
    
    flux_to_plot = BATMAN_MODEL(Baseline, Rp, t = times)
            
    if plot:
        plt.errorbar(data_times,flux,yerr=error,fmt='o',alpha = 0.5,label='Data')
        plt.plot(times,flux_to_plot,label='Rp/R* = {a:.2f}'.format(a=Rp),color='k')
        plt.legend(frameon=False)
        plt.xlabel('Time (hr from mid transit)')
        plt.ylabel('Relative Flux')
        plt.title(planet)
        plt.savefig('Gaussian_Fit_{}.pdf'.format(planet))
        plt.show()
        

    if (0.0 < Baseline) and (0.0 <= Rp < 1.5):
        
        lnp = np.sum(-(1/2)*(data-model)**2.0/(error)**2.0)
        
        return lnp
        if lnp == NaN:
            lnp = -np.inf
    
    return -np.inf

In [ ]:
def plot_chain(start=0, stop=nsteps):
    '''Plot the chain, in a couple different ways.'''

    Base, Rp = sampler.chain.T
    Base_trimmed, Rp_trimmed = sampler.chain.T[:, start:stop, :]

    plt.figure(figsize=(14,14))
    gs = plt.matplotlib.gridspec.GridSpec(3,2, hspace=0.6)
    
    #Walker Plots
    ax_Base = plt.subplot(gs[0,0])
    ax_Base.plot(Base.flatten()[::nwalkers],color='black',alpha=0.5); 
    ax_Base.axvspan(start, stop, zorder=-1,alpha=0.3); 
    ax_Base.set_ylabel('Baseline Flux')
    
    ax_Rp = plt.subplot(gs[0,1], sharex=ax_Base)
    ax_Rp.plot(Rp.flatten()[::nwalkers],color='black',alpha=0.5)
    ax_Rp.axvspan(start, stop, zorder=-1,alpha=0.3)
    ax_Rp.set_ylabel('Rp')
    
    #Histograms
    ax_Basehist = plt.subplot(gs[1,0])
    ax_Basehist.hist(Base.flatten(),color='black',bins=60)
    #ax_Basehist.axvline(1.5,zorder=100,color='purple')
    ax_Basehist.set_xlabel('Baseline')
    ax_Basehist.set_xlim(0,10)
    
    ax_Rphist = plt.subplot(gs[1,1])
    ax_Rphist.hist(Rp.flatten(),color='black',bins=60)
    #ax_Rphist.axvline(0.15,zorder=100,color='purple')
    ax_Rphist.set_xlabel('Rp')
    
    #Scatter Plots
    ax_both = plt.subplot(gs[2,0])
    ax_both.scatter(Base_trimmed, Rp_trimmed, s=5, alpha=0.2)
    ax_both.set_title('{} to {}'.format(start, stop))
    ax_both.set_xlabel('Baseline')
    ax_both.set_ylabel('Rp')

In [ ]:
date,flux = DataPrep(dir_inp,planet)
error_estimates = Fast_Fit(date,flux)
run_emcee(planet)
plot_chain()